In [1]:
# Dependencies
from bs4 import BeautifulSoup as bs
import requests
import pymongo
from splinter import Browser
import pandas as pd
from flask import Flask, render_template
from webdriver_manager.chrome import ChromeDriverManager
import time

In [2]:
    executable_path = {'executable_path': ChromeDriverManager().install()}
    browser = Browser('chrome', **executable_path, headless=False)



====== WebDriver manager ======
Current google-chrome version is 92.0.4515
Get LATEST driver version for 92.0.4515
There is no [mac64] chromedriver for browser 92.0.4515 in cache
Get LATEST driver version for 92.0.4515
Trying to download new driver from https://chromedriver.storage.googleapis.com/92.0.4515.43/chromedriver_mac64.zip
Driver has been saved in cache [/Users/staceydougherty/.wdm/drivers/chromedriver/mac64/92.0.4515.43]


In [3]:
#visit site
url = "https://redplanetscience.com/"
browser.visit(url)

time.sleep(1)

In [4]:
# Scrape page into Soup
html = browser.html
soup = bs(html, "html.parser")
#soup

In [8]:
# Get the news name
news_title = soup.find_all('div', class_='content_title')[0].text

# Get the news paragraph
news_p = soup.find_all('div', class_='article_teaser_body')[0].text
#print it out to see if it found it
news_title
news_p

'After the rover was shipped from JPL to Kennedy Space Center, the team is getting closer to finalizing the spacecraft for launch later this summer.'

In [10]:
#visit site for picture
featured_image_url = "https://spaceimages-mars.com/"
browser.visit(featured_image_url)
    
time.sleep(1)
    
html = browser.html
soup = bs(html, "html.parser")
soup

<html class=""><head>
<meta charset="utf-8"/>
<meta content="width=device-width, initial-scale=1" name="viewport"/>
<link href="https://maxcdn.bootstrapcdn.com/bootstrap/4.0.0/css/bootstrap.min.css" rel="stylesheet"/>
<!-- <link rel="stylesheet" type="text/css" href="css/font.css"> -->
<link href="css/app.css" rel="stylesheet" type="text/css"/>
<link href="https://stackpath.bootstrapcdn.com/font-awesome/4.7.0/css/font-awesome.min.css" rel="stylesheet" type="text/css"/>
<title>Space Image</title>
<style type="text/css">.fancybox-margin{margin-right:0px;}</style></head>
<body>
<div class="header">
<nav class="navbar navbar-expand-lg">
<a class="navbar-brand" href="#"><img id="logo" src="image/nasa.png"/><span class="logo">Jet Propulsion Laboratory</span>
<span class="logo1">California Institute of Technology</span></a>
<button aria-controls="navbarNav" aria-expanded="false" aria-label="Toggle navigation" class="navbar-toggler" data-target="#navbarNav" data-toggle="collapse" type="button"

In [11]:
relative_image_path = soup.find_all('img')[1]["src"]
featured_img_url = featured_image_url + relative_image_path
    
featured_img_url

'https://spaceimages-mars.com/image/featured/mars2.jpg'

In [12]:
#mars facts scraping
mars_facts_url = 'https://galaxyfacts-mars.com/'
tables = pd.read_html(mars_facts_url)
#tables

table_df = pd.DataFrame(tables[1])
table_df.columns = ["Attributes", "Values"]
table_df.set_index("Attributes", inplace =True)
table_df
#convert to html table
html_table = table_df.to_html()
html_table

html_table = html_table.replace('\n', '')
print(html_table)

<table border="1" class="dataframe">  <thead>    <tr style="text-align: right;">      <th></th>      <th>Values</th>    </tr>    <tr>      <th>Attributes</th>      <th></th>    </tr>  </thead>  <tbody>    <tr>      <th>Equatorial Diameter:</th>      <td>6,792 km</td>    </tr>    <tr>      <th>Polar Diameter:</th>      <td>6,752 km</td>    </tr>    <tr>      <th>Mass:</th>      <td>6.39 × 10^23 kg (0.11 Earths)</td>    </tr>    <tr>      <th>Moons:</th>      <td>2 ( Phobos &amp; Deimos )</td>    </tr>    <tr>      <th>Orbit Distance:</th>      <td>227,943,824 km (1.38 AU)</td>    </tr>    <tr>      <th>Orbit Period:</th>      <td>687 days (1.9 years)</td>    </tr>    <tr>      <th>Surface Temperature:</th>      <td>-87 to -5 °C</td>    </tr>    <tr>      <th>First Record:</th>      <td>2nd millennium BC</td>    </tr>    <tr>      <th>Recorded By:</th>      <td>Egyptian astronomers</td>    </tr>  </tbody></table>


In [13]:
#mars hemisphere
mars_url = 'https://marshemispheres.com/'
browser.visit(mars_url)
html=browser.html
soup=bs(html,'html.parser')

hemisphere_image_urls = []

for i in range(4):
    #create empty dictionary
    hemispheres = {}
    browser.find_by_css('a.product-item h3')[i].click()
    element = browser.links.find_by_text('Sample').first
    img_url = element['href']
    title = browser.find_by_css("h2.title").text
    hemispheres["img_url"] = img_url
    hemispheres["title"] = title
    hemisphere_image_urls.append(hemispheres)
    browser.back()
    


In [14]:
print(hemisphere_image_urls)

[{'img_url': 'https://marshemispheres.com/images/full.jpg', 'title': 'Cerberus Hemisphere Enhanced'}, {'img_url': 'https://marshemispheres.com/images/schiaparelli_enhanced-full.jpg', 'title': 'Schiaparelli Hemisphere Enhanced'}, {'img_url': 'https://marshemispheres.com/images/syrtis_major_enhanced-full.jpg', 'title': 'Syrtis Major Hemisphere Enhanced'}, {'img_url': 'https://marshemispheres.com/images/valles_marineris_enhanced-full.jpg', 'title': 'Valles Marineris Hemisphere Enhanced'}]


In [15]:
browser.quit()

In [16]:
    mars_scraped_data = {
        "news_titles": news_title,
        "news_p": news_p,
        "featuredimage_title": featured_image_url,
        "mars_facts_table": html_table,
        "hemisphere_images": hemisphere_image_urls
    }
mars_scraped_data

{'news_titles': "NASA's Perseverance Mars Rover Gets Its Wheels and Air Brakes",
 'news_p': 'After the rover was shipped from JPL to Kennedy Space Center, the team is getting closer to finalizing the spacecraft for launch later this summer.',
 'featuredimage_title': 'https://spaceimages-mars.com/',
 'mars_facts_table': '<table border="1" class="dataframe">  <thead>    <tr style="text-align: right;">      <th></th>      <th>Values</th>    </tr>    <tr>      <th>Attributes</th>      <th></th>    </tr>  </thead>  <tbody>    <tr>      <th>Equatorial Diameter:</th>      <td>6,792 km</td>    </tr>    <tr>      <th>Polar Diameter:</th>      <td>6,752 km</td>    </tr>    <tr>      <th>Mass:</th>      <td>6.39 × 10^23 kg (0.11 Earths)</td>    </tr>    <tr>      <th>Moons:</th>      <td>2 ( Phobos &amp; Deimos )</td>    </tr>    <tr>      <th>Orbit Distance:</th>      <td>227,943,824 km (1.38 AU)</td>    </tr>    <tr>      <th>Orbit Period:</th>      <td>687 days (1.9 years)</td>    </tr>    <tr